In [802]:
from sympy import *

In [803]:
divVs, divqD, Pt0, Pf0, phi_0 = symbols('divVs, divqD, Pt0, Pf0, Φ0')
K_s, K_f, K_phi, eta_phi, sin_psi, gamma = symbols('Ks, Kf, KΦ, ηΦ, sinψ, λ̇')
Pt, Pf, dt = symbols('Pt, Pf, Δt')
phi = symbols('Φ_trial')

In [804]:
# K_f = K_phi
# K_s = K_phi


In [805]:
# divVs, divqD, Pt0, Pf0, phi_0 = symbols('divVs, divqD, Pt0, Pf0, phi_0')
# K_s, K_f, K_phi, eta_phi, sin_psi, gamma = symbols('K_s, K_f, K_phi, eta_phi, sin_psi, gamma')
# Pt, Pf, dt = symbols('Pt, Pf, dt')
# phi = symbols('phi')

In [806]:
elastic = 1
viscous = 1
plastic = 1
single_phase = 2

In [807]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt
dphidt = elastic/K_phi*(dPfdt-dPtdt) + viscous/eta_phi*(Pf - Pt)

if single_phase==1:
    dphidt = 0
    phi = 0


# fpt = divVs + (dPtdt-phi*dPfdt)/K_s/(1-phi) + ( (dPtdt-dPfdt)/K_phi + (Pt-Pf)/eta_phi - gamma*sin_psi)/(1-phi)
dlnrhosdt = elastic * 1/(K_s*(1-phi)) * (dPtdt - phi*dPfdt) 
fpt       = dlnrhosdt - dphidt/(1-phi) + divVs
fpt       = fpt.simplify()

# fpf = divqD + divVs*phi + gamma*sin_psi + (Pf-Pt)/eta_phi + (dPfdt - dPtdt)/K_phi + phi*dPfdt/K_f
dlnrhofdt = dPfdt/K_f
fpf       = elastic * phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf       = fpf.simplify()

fphi = phi    - (phi_0 + dphidt*dt)

display(fpt)
display(fpf)

(Ks*KΦ*divVs*Δt*ηΦ*(Φ_trial - 1) + Ks*(KΦ*Δt*(Pf - Pt) + ηΦ*(Pf - Pf0 - Pt + Pt0)) + KΦ*ηΦ*(-Pt + Pt0 + Φ_trial*(Pf - Pf0)))/(Ks*KΦ*Δt*ηΦ*(Φ_trial - 1))

(Kf*KΦ*Δt*ηΦ*(divVs*Φ_trial + divqD) + Kf*KΦ*Δt*(Pf - Pt) + Kf*ηΦ*(Pf - Pf0 - Pt + Pt0) + KΦ*Φ_trial*ηΦ*(Pf - Pf0))/(Kf*KΦ*Δt*ηΦ)

In [808]:
P_trial = solve([fpt, fpf], [Pt, Pf])

In [809]:
julia_code(P_trial[Pt].simplify())

'(-Kf .* Ks .* KΦ .* divVs .* Δt .^ 2 - Kf .* Ks .* KΦ .* divqD .* Δt .^ 2 - Kf .* Ks .* divVs .* Δt .* ηΦ - Kf .* Ks .* divqD .* Δt .* ηΦ - Kf .* KΦ .* Pf0 .* Δt .* Φ_trial + Kf .* KΦ .* Pt0 .* Δt - Kf .* KΦ .* divVs .* Δt .* Φ_trial .^ 2 .* ηΦ - Kf .* KΦ .* divqD .* Δt .* Φ_trial .* ηΦ - Kf .* Pt0 .* Φ_trial .* ηΦ + Kf .* Pt0 .* ηΦ + Ks .* KΦ .* Pf0 .* Δt .* Φ_trial + Ks .* KΦ .* divVs .* Δt .* Φ_trial .^ 2 .* ηΦ - Ks .* KΦ .* divVs .* Δt .* Φ_trial .* ηΦ + Ks .* Pt0 .* Φ_trial .* ηΦ + KΦ .* Pt0 .* Φ_trial .* ηΦ) ./ (-Kf .* KΦ .* Δt .* Φ_trial + Kf .* KΦ .* Δt - Kf .* Φ_trial .* ηΦ + Kf .* ηΦ + Ks .* KΦ .* Δt .* Φ_trial + Ks .* Φ_trial .* ηΦ + KΦ .* Φ_trial .* ηΦ)'

In [810]:
if single_phase==0:
    julia_code(P_trial[Pf].simplify())

In [811]:
if single_phase==0:
    julia_code(solve(fphi, phi)[0] .simplify())

In [ ]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt

# phi    = phi_0 + dphidt*dt

if single_phase==1:
    dphidt = plastic*gamma*sin_psi
else:
    dphidt = elastic/K_phi*(dPfdt-dPtdt) + viscous/eta_phi*(Pf - Pt) + plastic*gamma*sin_psi
    
dlnrhosdt = elastic/K_s/(1-phi) * (dPtdt - phi*dPfdt) + 0*1/(1-phi)*plastic*gamma*sin_psi
fpt = dlnrhosdt - dphidt/(1-phi) + divVs
fpt = fpt.simplify()

dlnrhofdt = dPfdt/K_f - 0*plastic*gamma*sin_psi
fpf  = elastic*phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf = fpf.simplify()

if single_phase==1:     
    P_corr = solve([fpt], [Pt])
else:
    P_corr = solve([fpt, fpf], [Pt, Pf])
P_corr

{Pt: (Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 - Kf*Ks*KΦ*sinψ*Δt**2*Φ_trial*λ̇ + Kf*Ks*KΦ*sinψ*Δt**2*λ̇ + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ - Kf*Ks*sinψ*Δt*Φ_trial*ηΦ*λ̇ + Kf*Ks*sinψ*Δt*ηΦ*λ̇ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Kf*KΦ*divqD*Δt*Φ_trial*ηΦ - Kf*KΦ*sinψ*Δt*Φ_trial**2*ηΦ*λ̇ + Kf*Pt0*Φ_trial*ηΦ - Kf*Pt0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Ks*KΦ*divVs*Δt*Φ_trial*ηΦ + Ks*KΦ*sinψ*Δt*Φ_trial*ηΦ*λ̇ - Ks*Pt0*Φ_trial*ηΦ - KΦ*Pt0*Φ_trial*ηΦ)/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ),
 Pf: (Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 - Kf*Ks*KΦ*sinψ*Δt**2*Φ_trial*λ̇ + Kf*Ks*KΦ*sinψ*Δt**2*λ̇ + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ - Kf*Ks*sinψ*Δt*Φ_trial*ηΦ*λ̇ + Kf*Ks*sinψ*Δt*ηΦ*λ̇ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial*ηΦ + Kf*KΦ*divqD*Δt*ηΦ - Kf*KΦ*sinψ*Δt*Φ_trial*ηΦ*λ̇ + Kf*Pf0*Φ_trial*ηΦ - Kf*Pf0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*Pf0

In [813]:
P_trial[Pt]

(Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Kf*KΦ*divqD*Δt*Φ_trial*ηΦ + Kf*Pt0*Φ_trial*ηΦ - Kf*Pt0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Ks*KΦ*divVs*Δt*Φ_trial*ηΦ - Ks*Pt0*Φ_trial*ηΦ - KΦ*Pt0*Φ_trial*ηΦ)/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ)

In [814]:
dPt = ((P_corr[Pt] - P_trial[Pt])).simplify()
display(dPt)
print('ΔPt = ' +  julia_code(dPt))

sinψ*Δt*λ̇*(Kf*Ks*KΦ*Δt*Φ_trial - Kf*Ks*KΦ*Δt + Kf*Ks*Φ_trial*ηΦ - Kf*Ks*ηΦ + Kf*KΦ*Φ_trial**2*ηΦ - Ks*KΦ*Φ_trial*ηΦ)/(-Kf*KΦ*Δt*Φ_trial + Kf*KΦ*Δt - Kf*Φ_trial*ηΦ + Kf*ηΦ + Ks*KΦ*Δt*Φ_trial + Ks*Φ_trial*ηΦ + KΦ*Φ_trial*ηΦ)

ΔPt = sinψ .* Δt .* λ̇ .* (Kf .* Ks .* KΦ .* Δt .* Φ_trial - Kf .* Ks .* KΦ .* Δt + Kf .* Ks .* Φ_trial .* ηΦ - Kf .* Ks .* ηΦ + Kf .* KΦ .* Φ_trial .^ 2 .* ηΦ - Ks .* KΦ .* Φ_trial .* ηΦ) ./ (-Kf .* KΦ .* Δt .* Φ_trial + Kf .* KΦ .* Δt - Kf .* Φ_trial .* ηΦ + Kf .* ηΦ + Ks .* KΦ .* Δt .* Φ_trial + Ks .* Φ_trial .* ηΦ + KΦ .* Φ_trial .* ηΦ)


In [815]:
if single_phase==0:
    dPf = ((P_corr[Pf] - P_trial[Pf])).simplify()
    julia_code(dPf)
    print('ΔPf = ' +  julia_code(dPf))